In [1]:
# === test_ga_rd.ipynb cell ===

import os, sys, time, random
sys.path.insert(0, os.path.abspath('..'))  # add project root to PYTHONPATH

import pandas as pd
import numpy as np

# 1) Imports from your package
from ga_rd.algorithm    import run_ga
from ga_rd.Individual_RD import IndividualRD
from ga_rd.selection    import tournament_selection, rank_selection
from ga_rd.crossover    import uniform_crossover
from ga_rd.mutation     import swap_mutation, scramble_mutation, inversion_mutation

# 2) Load mock data
coords_df   = pd.read_csv(os.path.join('..', 'Data', 'coordinates_map.csv'), index_col='Index')
house_coords = coords_df[['Latitude','Longitude']].values[:10]

# 3) Create & inspect one IndividualRD
ind = IndividualRD(
    house_coords=house_coords,
    n_participants=18,
    capacity_of_houses=6,
    a=1.0,
    crossover_method='random',
    crossover_probability=0.8,
    mutation_method='swap',
    mutation_prob=0.5
)
ind.random_representation()
print("Initial genome:", ind.solution.genome)
print("Initial fitness:", ind.fitness, "\n")

# 4) Test selection operators
pop = [IndividualRD(house_coords,18,6,1.0) for _ in range(10)]
for i in pop: i.random_representation()
t_win = tournament_selection(pop, k=3)
r_win = rank_selection(pop)
print("Tournament winner fitness:", t_win.fitness)
print("Rank‐selection winner fitness:", r_win.fitness, "\n")

# 5) Test IndividualRD.crossover (wraps one-point/uniform)
ind2 = pop[1]
child_ind = ind.crossover(ind2)
print("Child (via IndividualRD.crossover) genome:", child_ind.solution.genome)
print("Child fitness:", child_ind.fitness, "\n")

# 6) Test IndividualRD.mutation
mut_ind = ind.mutation()
print("Mutated genome:", mut_ind.solution.genome)
print("Mutated fitness:", mut_ind.fitness, "\n")

# 7) Test standalone crossover on SolutionRD
print("=== Standalone crossover (SolutionRD) ===")
c1 = one_point_crossover(ind.solution, ind2.solution)
print(" One-point child genome:", c1.genome)
c2 = uniform_crossover(ind.solution, ind2.solution, verbose=True)
print(" Uniform child genome:", c2.genome)
c3 = block_swap_crossover(ind.solution, ind2.solution, verbose=True)
print(" Block-swap child genome:", c3.genome, "\n")

# 8) Test standalone mutation via wrappers
print("=== Standalone mutations ===")
sm = swap_mutation(ind)
print(" Swap mutation genome:", sm.solution.genome)
sc = scramble_mutation(ind)
print(" Scramble mutation genome:", sc.solution.genome)
iv = inversion_mutation(ind)
print(" Inversion mutation genome:", iv.solution.genome, "\n")

# 9) Run a mini‐GA
print("=== Running GA… ===")
t0 = time.perf_counter()
final_pop = run_ga(
    pop_size=20,
    house_coords=house_coords,
    n_participants=18,
    capacity=6,
    a=1.0,
    generations=3,
    selection_method='tournament',
    tournament_size=3,
    crossover_method='random'
)
dt = time.perf_counter() - t0
best = min(final_pop, key=lambda x: x.fitness)
print(f"GA finished in {dt:.2f}s; best fitness = {best.fitness:.3f}")
print("Best genome:", best.solution.genome)


ImportError: cannot import name 'IndividualRD' from partially initialized module 'ga_rd.Individual_RD' (most likely due to a circular import) (/Users/zofiawojcik/Gen_Alg1_RunningDinner/ga_rd/Individual_RD.py)